In [1]:
import os
import numpy as np
from deepface import DeepFace
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import pandas as pd

2024-06-03 02:25:26.796235: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model_names = [
    "VGG-Face",
    "Facenet",
    "Facenet512",
    "OpenFace",
    "DeepFace",
    "DeepID",
    "ArcFace",
    "SFace",
    "GhostFaceNet"
]

vertical_folders = ["up", "center", "down"]
horizontal_folders = ["45_left", "center", "45_right"]

dataset_path = "/tf/data/dataset_clean/"
people_list = [person for person in os.listdir(os.path.join(dataset_path, "center/center")) if not person.startswith(".")]
print(people_list)
print(len(people_list))

['aabatashova', 'aamartynenko_9', 'aapardaev', 'aaraimova', 'aazheltov_1', 'adletunov', 'adpiskunov', 'aimilovanova', 'akklochkova', 'ameliseenko', 'amsirotkin', 'amzayakina', 'anguen', 'ankarabutova', 'appecherskih', 'arandarbua', 'archilingarov', 'arkhabibullina_3', 'asbaranova', 'aschinkov', 'assupolyarina', 'avkarpukhin_1', 'avpanarin', 'avpapina', 'avvorobev_5', 'ayuvasileva_1', 'baserebrov', 'dadmitriev', 'dashubin', 'datikhomirov', 'demirkin', 'deredin_1', 'didiulgerov', 'dkorolev', 'dmbobrova', 'drsmirnov', 'dvzaitseva', 'eabykova_6', 'eamilekhina', 'eelogvina', 'eggadzhiev', 'emdanilov', 'evdarkhanov', 'gaataev', 'gdpekert', 'gyuvasilev', 'ikmityushkin', 'imdolgov', 'imsalakhov', 'ipustenko', 'ischekanov', 'isulanova', 'ivlukianov', 'ivsemichasnov', 'kdfrolov', 'kdgryazeva', 'khtalbegov', 'kvnovikov', 'limiroshnichenko', 'madusov', 'mapivovarova', 'masadkovskaya', 'mdmilchakova', 'mdspitsyn', 'mismailzade', 'mrkalkanov_1', 'msbelyaeva_1', 'mvemelianenko', 'mvmoiseev', 'mvyakov

In [7]:
perplexities = [5, 7.5, 10, 12.5, 15]
n_iters = [1000, 2500, 5000, 7500, 10000]
learning_rates = [200, 400, 600, 800, 1000]

In [8]:
def plot_tsne(tsne, face_recognition_model, embedding_labels, suffix = ""):
    print(tsne.shape)
    lookupTable, indexed_dataSet = np.unique(embedding_labels, return_inverse=True)
    print(len(indexed_dataSet))
    print(np.unique(embedding_labels).shape)

    fig, ax = plt.subplots()
    
    scatter = ax.scatter(tsne[:, 0], tsne[:, 1], cmap='hsv', s=100, c=indexed_dataSet, edgecolors=['black'], linewidth=1)
    ax.set_aspect('equal', 'datalim')
    fig.set_size_inches(20, 15)

    colorbar = plt.colorbar(scatter, ax=ax, boundaries=np.arange(len(np.unique(embedding_labels))+1)-0.5)
    colorbar.set_ticks(np.arange(len(np.unique(embedding_labels))))
    colorbar.ax.set_yticklabels(np.unique(embedding_labels))
    
    for i, txt in zip(tsne, embedding_labels):
        ax.annotate(txt, (i[0], i[1]), fontsize=5, rotation=45)

    ax.set_title(f't-SNE on {face_recognition_model} embeddings', fontsize=24)
    ax.set_xticks(np.linspace(-6, 6, 7))
    ax.set_yticks(np.linspace(-6, 6, 7))

    plt.tight_layout()
    plt.savefig(f"best_t-sne_{face_recognition_model}_all_angles{suffix}.png", dpi=150)
    plt.close(fig)

In [ ]:
param_grid = [(perp, n_iter, lr) for perp in perplexities for n_iter in n_iters for lr in learning_rates]

# считаем метрику silhouette_score
def calculate_silhouette_score(embeddings, labels):
    score = silhouette_score(embeddings, labels, metric='euclidean')
    return score

results_df = pd.DataFrame(columns=['model', 'perplexity', 'iterations', 'learning rate', 'silhouette score'])

for face_recognition_model in model_names:
    face_embeddings = []
    embedding_labels = []
    print(f"model: {face_recognition_model}")
    
    for vertical_folder_name in vertical_folders:
        vertical_folder_path = os.path.join(dataset_path, vertical_folder_name)
        os.makedirs(vertical_folder_path, exist_ok=True)
    
        for horizontal_folder_name in horizontal_folders:
            horizontal_folder_path = os.path.join(vertical_folder_path, horizontal_folder_name)
            os.makedirs(horizontal_folder_path, exist_ok=True)
            for person in tqdm(people_list):
                person_directory_path = os.path.join(horizontal_folder_path, person)
                for image in [image for image in os.listdir(person_directory_path) if image.endswith(('.JPG', '.jpg'))]:
                    image_path = os.path.join(person_directory_path, image)
                    
                    embedding = DeepFace.represent(
                        img_path=image_path, 
                        model_name=face_recognition_model,
                        detector_backend='retinaface',
                        enforce_detection=False
                    )
                    face_embeddings.append(embedding[0]['embedding'])
                    embedding_labels.append(person)
    
    face_embeddings = np.asarray(face_embeddings)
    print(f"embeddings array formed, its shape is {face_embeddings.shape}")
    
    best_score = -1
    best_params = None
    best_tsne = None

    for perp, n_iter, lr in param_grid:
        print(f"running t-SNE with perplexity={perp}, n_iter={n_iter}, learning_rate={lr}")
        tsne_embeddings = TSNE(random_state=42, n_components=2, verbose=1, perplexity=perp, n_iter=n_iter, learning_rate=lr).fit_transform(face_embeddings)
        score = calculate_silhouette_score(tsne_embeddings, embedding_labels)
        new_row = pd.DataFrame({
            'model': [face_recognition_model],
            'perplexity': [perp],
            'iterations': [n_iter],
            'learning rate': [lr],
            'silhouette score': [score]
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

        if score > best_score:
            best_score = score
            best_params = (perp, n_iter, lr)
            best_tsne = tsne_embeddings

    best_perp, best_n_iter, best_lr = best_params
    print(f"best parameters for {face_recognition_model}: perplexity={best_perp}, n_iter={best_n_iter}, learning_rate={best_lr}, silhouette Score={best_score}")
    plot_tsne(best_tsne, face_recognition_model, embedding_labels, suffix=f"_{best_perp}_{best_n_iter}_{best_lr}")

results_df.to_csv("tsne_params.csv", index=False)

model: VGG-Face


100%|██████████| 100/100 [01:08<00:00,  1.45it/s]


embeddings array formed, its shape is (1188, 4096)
running t-SNE with perplexity=5, n_iter=1000, learning_rate=200
[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Indexed 1188 samples in 0.002s...
[t-SNE] Computed neighbors for 1188 samples in 0.104s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1188
[t-SNE] Computed conditional probabilities for sample 1188 / 1188
[t-SNE] Mean sigma: 0.278567
[t-SNE] KL divergence after 250 iterations with early exaggeration: 86.067886
[t-SNE] KL divergence after 1000 iterations: 1.008097
running t-SNE with perplexity=5, n_iter=1000, learning_rate=400
[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Indexed 1188 samples in 0.003s...
[t-SNE] Computed neighbors for 1188 samples in 0.100s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1188
[t-SNE] Computed conditional probabilities for sample 1188 / 1188
[t-SNE] Mean sigma: 0.278567
[t-SNE] KL divergence after 250 iterations with early exaggeration: 92.414368
[t-SNE]

100%|██████████| 100/100 [01:30<00:00,  1.11it/s]


embeddings array formed, its shape is (1188, 128)
running t-SNE with perplexity=5, n_iter=1000, learning_rate=200
[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Indexed 1188 samples in 0.000s...
[t-SNE] Computed neighbors for 1188 samples in 0.026s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1188
[t-SNE] Computed conditional probabilities for sample 1188 / 1188
[t-SNE] Mean sigma: 2.123365
[t-SNE] KL divergence after 250 iterations with early exaggeration: 88.367554
[t-SNE] KL divergence after 1000 iterations: 1.171474
running t-SNE with perplexity=5, n_iter=1000, learning_rate=400
[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Indexed 1188 samples in 0.000s...
[t-SNE] Computed neighbors for 1188 samples in 0.069s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1188
[t-SNE] Computed conditional probabilities for sample 1188 / 1188
[t-SNE] Mean sigma: 2.123365
[t-SNE] KL divergence after 250 iterations with early exaggeration: 95.015259
[t-SNE] 

...

100%|██████████| 100/100 [01:07<00:00,  1.48it/s]


embeddings array formed, its shape is (1188, 160)
running t-SNE with perplexity=5, n_iter=1000, learning_rate=200
[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Indexed 1188 samples in 0.000s...
[t-SNE] Computed neighbors for 1188 samples in 0.023s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1188
[t-SNE] Computed conditional probabilities for sample 1188 / 1188
[t-SNE] Mean sigma: 13.298290
[t-SNE] KL divergence after 250 iterations with early exaggeration: 85.829674
[t-SNE] KL divergence after 1000 iterations: 1.196581
running t-SNE with perplexity=5, n_iter=1000, learning_rate=400
[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Indexed 1188 samples in 0.000s...
[t-SNE] Computed neighbors for 1188 samples in 0.071s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1188
[t-SNE] Computed conditional probabilities for sample 1188 / 1188
[t-SNE] Mean sigma: 13.298290
[t-SNE] KL divergence after 250 iterations with early exaggeration: 90.790115
[t-SNE

100%|██████████| 100/100 [01:16<00:00,  1.31it/s]


embeddings array formed, its shape is (1188, 512)
running t-SNE with perplexity=5, n_iter=1000, learning_rate=200
[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Indexed 1188 samples in 0.000s...
[t-SNE] Computed neighbors for 1188 samples in 0.029s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1188
[t-SNE] Computed conditional probabilities for sample 1188 / 1188
[t-SNE] Mean sigma: 0.677129
[t-SNE] KL divergence after 250 iterations with early exaggeration: 95.990730
[t-SNE] KL divergence after 1000 iterations: 1.493319
running t-SNE with perplexity=5, n_iter=1000, learning_rate=400
[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Indexed 1188 samples in 0.000s...
[t-SNE] Computed neighbors for 1188 samples in 0.077s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1188
[t-SNE] Computed conditional probabilities for sample 1188 / 1188
[t-SNE] Mean sigma: 0.677129
[t-SNE] KL divergence after 250 iterations with early exaggeration: 99.251411
[t-SNE] 

 77%|███████▋  | 77/100 [00:53<00:23,  1.03s/it]

In [31]:
!pip install scikit-learn

In [32]:
from sklearn.cluster import KMeans

ImportError: cannot import name '_deprecate_Xt_in_inverse_transform' from 'sklearn.utils.deprecation' (/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py)